# Scikit-Tuning Hyperparameter Estimator in Machine Learning

[Here](https://scikit-learn.org/stable/modules/grid_search.html)

Hyperparameter optimization refers to performing a search in order to discover the set of specific model configuration arguments that result in the best performance of the model on a specific dataset

## Tutorial Overview
This tutorial is divided into four parts; they are:

1. Scikit-Optimize
2. Machine Learning Dataset and Model
3. Manually Tune Algorithm Hyperparameters
4. Automatically Tune Algorithm Hyperparameters
5. Automatically Algorithms Hyperparameters Sample
    - 5.1 GridSearchCV 
    - 5.2 RandomizedSearchCV
    - 5.3 BayesSearchCV

## 1. Scikit-Optimize

In [ ]:
#user Anaconda PowerShell/console with Administrative User Rights
#conda install -c conda-forge scikit-optimize

In [2]:
# report scikit-optimize version number
import skopt
print('skopt %s' % skopt.__version__)

skopt 0.8.1


## 2. Machine Learning Dataset and Model
We will use the ionosphere machine learning dataset. This is a standard machine learning dataset comprising 351 rows of data with three numerical input variables and a target variable with two class values, e.g. binary classification.

Using a test harness of repeated stratified 10-fold cross-validation with three repeats, a naive model can achieve an accuracy of about 64 percent.

You can learn more about the dataset here:

Ionosphere Dataset ([ionosphere.csv](https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv))
Ionosphere Dataset Description ([ionosphere.names](https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.names))

In [3]:
# evaluate an svm for the ionosphere dataset
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC

# load dataset
#url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
url = '..//..//data//ionosphere.data.csv'
dataframe = read_csv(url, header=None)
dataframe.shape

(351, 35)

In [3]:
# split into input and output elements
data = dataframe.values
X, y = data[:, :-1], data[:, -1]
print(X.shape, y.shape)

NameError: name 'dataframe' is not defined

In [5]:
# define model model
model = SVC()

# define test harness
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# evaluate model
m_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(m_scores), std(m_scores)))

Accuracy: 0.937 (0.038)


## 3. Manually Tune Algorithm Hyperparameters
The Scikit-Optimize library can be used to tune the hyperparameters of a machine learning model.

We can achieve this manually by using the Bayesian Optimization capabilities of the library.

This requires that we first define a search space. In this case, this will be the hyperparameters of the model that we wish to tune, and the scope or range of each hyperparameter.

We will tune the following [hyperparameters of the SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) model:

- __C__, the regularization parameter.
- __kernel__, the type of kernel used in the model.
- __degree__, used for the polynomial kernel.
- __gamma__, used in most other kernels.

For the numeric hyperparameters __C__ and __gamma__, we will define a log scale to search between a small value of 1e-6 and 100. __Degree__ is an integer and we will search values between 1 and 5. Finally, the __kernel__ is a categorical variable with specific named values.

In [6]:
# manually tune svm model hyperparameters using skopt on the ionosphere dataset
from numpy import mean
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC

from skopt.space import Integer
from skopt.space import Real
from skopt.space import Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize

In [7]:
# define the space of hyperparameters to search
search_space = list()
search_space.append(Real(1e-6, 100.0, 'log-uniform', name='C'))
search_space.append(Categorical(['linear', 'poly', 'rbf', 'sigmoid'], name='kernel'))
search_space.append(Integer(1, 5, name='degree'))
search_space.append(Real(1e-6, 100.0, 'log-uniform', name='gamma'))

Note the data type, the range, and the name of the hyperparameter specified for each.

We can then define a function that will be called by the search procedure. This is a function expected by the optimization procedure later and takes a model and set of specific hyperparameters for the model, evaluates it, and returns a score for the set of hyperparameters.

In our case, we want to evaluate the model using repeated stratified 10-fold cross-validation on our ionosphere dataset. We want to maximize classification accuracy, e.g. find the set of model hyperparameters that give the best accuracy. By default, the process minimizes the score returned from this function, therefore, we will return one minus the accuracy, e.g. perfect skill will be (1 – accuracy) or 0.0, and the worst skill will be 1.0.

The evaluate_model() function below implements this and takes a specific set of hyperparameters.

In [8]:
# define the function used to evaluate a given configuration
@use_named_args(search_space)
def evaluate_model(**params):
    # configure the model with specific hyperparameters
    model = SVC()
    model.set_params(**params)

    # define test harness
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    
    # calculate 5-fold cross validation
    result = cross_val_score(model, X, y, cv=cv, n_jobs=-1, scoring='accuracy')
    
    # calculate the mean of the scores
    estimate = mean(result)
    
    # convert from a maximizing score to a minimizing score
    return 1.0 - estimate

In [2]:
# load dataset
# url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
url = '..//..//data//ionosphere.data.csv'
dataframe = read_csv(url, header=None)

# split into input and output elements
data = dataframe.values
X, y = data[:, :-1], data[:, -1]
print(X.shape, y.shape)

NameError: name 'read_csv' is not defined

Next, we can execute the search by calling the gp_minimize() function and passing the name of the function to call to evaluate each model and the search space to optimize.

In [9]:
# perform optimization
result = gp_minimize(evaluate_model, search_space)

# summarizing finding:
print('Best Accuracy: %.3f' % (1.0 - result.fun))

(351, 34) (351,)
Best Accuracy: 0.950
Best Parameters: [2.4835638090682264, 'rbf', 1, 0.11662917203714443]


The procedure will run until it converges and returns a result.

The result object contains lots of details, but importantly, we can access the score of the best performing configuration and the hyperparameters used by the best forming model. 

In [ ]:
print('Best Parameters: (C, kernel, degree, gamma) %s' % (result.x))

## 4. Automatically Tune Algorithm Hyperparameters

The Scikit-Learn machine learning library provides tools for tuning model hyperparameters.

Specifically, it provides the [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) and [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) classes that take a model, a search space, and a cross-validation configuration.

The benefit of these classes is that the search procedure is performed automatically, requiring minimal configuration.

Similarly, the Scikit-Optimize library provides a similar interface for performing a Bayesian Optimization of model hyperparameters via the [BayesSearchCV](https://scikit-optimize.github.io/modules/generated/skopt.BayesSearchCV.html) class.

In [ ]:
# automatic svm hyperparameter tuning using skopt for the ionosphere dataset
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV

# load dataset
# url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
url = '..//..//data//ionosphere.data.csv'
dataframe = read_csv(url, header=None)

# split into input and output elements
data = dataframe.values
X, y = data[:, :-1], data[:, -1]
print(X.shape, y.shape)

In [ ]:
# define search space
params = dict()
params['C'] = (1e-6, 100.0, 'log-uniform')
params['gamma'] = (1e-6, 100.0, 'log-uniform')
params['degree'] = (1,5)
params['kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']

In [ ]:
# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# define the search
search = BayesSearchCV(estimator=SVC(), search_spaces=params, n_jobs=-1, cv=cv)

# perform the search
search.fit(X, y)

# report the best result
print(search.best_score_)
print(search.best_params_)

## 5. Automatically Algorithms Hyperparameters Sample
### 5.1 GridSearchCV 

In [39]:
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

X, y = load_iris(True)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=0)

# log-uniform: understand as search over p = exp(x) by varying x
searchGS = GridSearchCV(
    estimator=SVC(),
    param_grid={
        'C': [1, 10, 100, 1000],
        'gamma': [0.001, 0.0001],
        'degree': [1, 8],
        'kernel': ('linear', 'poly', 'rbf'),
    }
)

# executes bayesian optimization
_ = searchGS.fit(X_train, y_train)

# model can be saved, used for predictions or scoring
print(searchGS.score(X_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass return_X_y=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


0.9736842105263158


In [40]:
# report the best result
print(searchGS.best_score_)
print(searchGS.best_params_)

0.9731225296442687
{'C': 1, 'degree': 1, 'gamma': 0.001, 'kernel': 'linear'}


### 5.2 RandomizedSearchCV

In [44]:
from sklearn.model_selection import RandomizedSearchCV

from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils.fixes import loguniform

X, y = load_iris(True)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=0)

# log-uniform: understand as search over p = exp(x) by varying x
searchRS = RandomizedSearchCV(
    estimator = SVC(),
    param_distributions = {
        'C': loguniform(1e0, 1e3),
        'gamma': loguniform(1e-4, 1e-3),
        'kernel': ['linear', 'poly', 'rbf'],
        'class_weight': ['balanced', None]
    },
    n_iter=32,
    random_state=0
)

# executes bayesian optimization
_ = searchRS.fit(X_train, y_train)

# model can be saved, used for predictions or scoring
print(searchRS.score(X_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass return_X_y=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


0.9736842105263158


In [45]:
# report the best result
print(searchRS.best_score_)
print(searchRS.best_params_)

0.9727272727272727
{'C': 9.03039728888593, 'class_weight': None, 'gamma': 0.0006955719813458962, 'kernel': 'linear'}


### 5.1 BayesSearchCV

In [24]:
from skopt import BayesSearchCV

# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer

from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

X, y = load_iris(True)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=0)

# log-uniform: understand as search over p = exp(x) by varying x
searchBS = BayesSearchCV(
    estimator=SVC(),
    search_spaces={
        'C': Real(1e-6, 1e+6, prior='log-uniform'),
        'gamma': Real(1e-6, 1e+1, prior='log-uniform'),
        'degree': Integer(1,8),
        'kernel': Categorical(['linear', 'poly', 'rbf']),
    },
    n_iter=32,
    random_state=0
)

# executes bayesian optimization
_ = searchBS.fit(X_train, y_train)

# model can be saved, used for predictions or scoring
print(searchBS.score(X_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass return_X_y=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


0.9736842105263158


In [25]:
# report the best result
print(searchBS.best_score_)
print(searchBS.best_params_)

0.9821428571428571
OrderedDict([('C', 1.3361910455737007), ('degree', 5), ('gamma', 0.11283439533114079), ('kernel', 'linear')])
